In [1]:
import os, re, lxml, requests, ee, geemap
import pandas as pd
import geopandas as gpd
from bs4 import BeautifulSoup
from shapely.geometry import Point
from datetime import date, datetime, time, timedelta

# Функциональность

In [2]:
def GetPageData(url):
    response = requests.get(url)
    response.encoding = 'utf-8'
    r = response.text
    soup = BeautifulSoup(r, 'lxml')
    return soup

def LinksByRegion(url):
    soup = GetPageData(url)
    li_list = soup.find_all('li', class_ = 'big-blue-billet__list_link')
    a_list = [li.find_all('a')[0] for li in li_list]
    href_list = [a.get('href') for a in a_list]
    links = [f'{os.path.dirname(url)}{link}' for link in href_list if len(re.findall(r'weather.php', link)) == 1]
    return links

def GetPointData(link): 
    soup = GetPageData(link)
     
    #extract point info
    point_info = soup.find_all("div", class_ = "archive-text")[0].find_all('span')
    
    point_name = point_info[0].text.split('(')[0][:-1]
    point_coords = Point(float(point_info[2].text), float(point_info[1].text))

    #extract point temperature
    try:
        point_temp = '{}°C'.format(float(soup.body.find_all('nobr')[-6].text))
    except:
        point_temp = None

    #extract point time
    try:
        date_ = soup.table.find_all('td', class_ = 'black')[-1].text
        time_ = soup.table.find_all('td', class_ = 'black')[-2].text
        point_date = date(datetime.now().timetuple()[0], int(date_.split('.')[1]), int(date_.split('.')[0]))
        point_time = time(int(time_))
        point_fulltime = datetime.combine(point_date, point_time)
    except:
        point_fulltime = None
        
    df_point = {'name': point_name , 'temperature': point_temp, 'time': point_fulltime, 'geometry': point_coords}
    return df_point

def GetRegionData(url):
    gdf_dict = {'name': [] , 'temperature': [], 'time': [], 'geometry': []}
    gdf = gpd.GeoDataFrame(gdf_dict, crs=4326)
    links = LinksByRegion(url)
    for link in links:
        df_point = GetPointData(link)
        gdf = gdf.append(df_point, ignore_index = True)
    return gdf

def ProcessRegionData(autocode): 
    start_time = datetime.now().timestamp()
    url = f'http://www.pogodaiklimat.ru/archive.php?id=ru&region={autocode}'
    gdf_region_ = GetRegionData(url)
    gdf_region = gdf_region_.set_crs(4326)
    finish_time = datetime.now().timestamp()
    print(f'Время сбора данных: {round(finish_time - start_time,2)} с')
    return gdf_region

def ProcessRegionDataInterface():
    autocode_source = 'http://www.consultant.ru/document/cons_doc_LAW_108669/88a12659e7cc781c56303430d98ae6c8a683892a/'
    print(f'Ссылка на таблицу кодов регионов: {autocode_source}')
    autocode = input('Введите код региона в соответствии с таблицей >>> ')
    try:
        print('Производится сбор данных...')
        gdf_region = ProcessRegionData(autocode)
    except:
        raise RuntimeError('Код введён неверно')
    if len(gdf_region) == 0:
        print('Код введён неверно')
    else:
        pass
    return gdf_region

def TimeQueryResult():
    time_sort = input('Хотите ли вы составить карту на определённый момент времени (Да/Нет)? >>> ').lower()
    if time_sort == 'да':
        print('Пример ввода времемни: 1970-01-01 00:00:00')
        time_in = input('Введите время в соответствии с таблицей (см. пример) >>> ')
        time_in_timestamp = pd.Timestamp(time_in)
        gdf_region_selection_query = gdf_region.query("time == @time_in_timestamp")
        gdf_region_selection = gdf_region_selection_query[['name', 'temperature', 'geometry']]
    elif time_sort == 'нет':
        gdf_region_selection = gdf_region[['name', 'temperature', 'geometry']]
    else:
        gdf_region_selection = TimeQueryResult()
    return gdf_region_selection
def AddMap():
    gdf_region_selection = TimeQueryResult()
    ee.Initialize()
    gee_region = geemap.geopandas_to_ee(gdf_region_selection)
    Map = geemap.Map()
    #points
    Map.add_gdf(gdf_region_selection, layer_name='Температуры', zoom_to_layer=True, info_mode = 'on_hover')
    Map.add_labels(gee_region, "temperature", font_size="10pt", font_color="blue", font_family="arial", font_weight="bold")
    return Map

# Варианты запуска

# Сбор данных

In [7]:
gdf_region = ProcessRegionDataInterface()
print(gdf_region[['name', 'temperature', 'time']])

Ссылка на таблицу кодов регионов: http://www.consultant.ru/document/cons_doc_LAW_108669/88a12659e7cc781c56303430d98ae6c8a683892a/
Введите код региона в соответствии с таблицей >>> 59
Производится сбор данных...
Время сбора данных: 11.62 с
               name temperature                 time
0         Березники     -11.0°C  2022-12-26 09:00:00
1             Бисер     -13.0°C  2022-12-26 15:00:00
2   Большая Соснова      -9.0°C  2022-12-25 06:00:00
3    Большое Савино     -10.0°C  2022-12-26 16:00:00
4               Вая      -3.0°C  2022-12-25 06:00:00
5        Верещагино     -12.0°C  2022-12-26 09:00:00
6             Гайны     -11.0°C  2022-12-26 12:00:00
7            Губаха      -9.0°C  2022-12-26 09:00:00
8          Добрянка      -9.0°C  2022-12-25 12:00:00
9              Коса     -11.0°C  2022-12-26 15:00:00
10           Кочево     -10.0°C  2022-12-26 15:00:00
11         Кудымкар      -9.0°C  2022-12-26 12:00:00
12           Кунгур     -13.0°C  2022-12-26 09:00:00
13              Кын

# Создание интерактивного слоя

In [8]:
AddMap()

Хотите ли вы составить карту на определённый момент времени (Да/Нет)? >>> да
Пример ввода времемни: 1970-01-01 00:00:00
Введите время в соответствии с таблицей (см. пример) >>> 2022-12-26 12:00:00


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…